In [1]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1

# config

In [2]:
from torch._C import device
device = "cuda"

BATCH_SIZE = 450
TEST_BATCH_SIZE = 450
TRAIN_FILE = '../input/riiid-test-answer-prediction/train.csv'
VAL_RATE = 0.01
MAX_EPOCHS = 5

MAX_SEQ = 100
EMBED_DIMS = 128
ENC_HEADS = DEC_HEADS = 8
NUM_ENCODER = NUM_DECODER = 6
TOTAL_EXE = TOTAL_RESP = 13523
TOTAL_CAT = 13523

# train data

In [3]:
import gc
import datatable as dt
import numpy as np
import pandas as pd
 
from torch.utils.data import Dataset, DataLoader
import gc
from sklearn.model_selection import train_test_split 


class Dataset(Dataset):
  def __init__(self,group,n_skills,max_seq = 100):
    self.samples = group
    self.n_skills = n_skills
    self.max_seq = max_seq
    self.data = []

    for que,ans,res_time,exe_cat in self.samples:
        if len(que)>=self.max_seq:
            self.data.extend([(que[l:l+self.max_seq],ans[l:l+self.max_seq],res_time[l:l+self.max_seq],exe_cat[l:l+self.max_seq])\
            for l in range(len(que)) if l%self.max_seq==0])
        elif len(que)<self.max_seq and len(que)>10:
            self.data.append((que,ans,res_time,exe_cat))
        else :
            continue
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,idx):
    content_ids,answered_correctly,response_time,exe_category = self.data[idx]
    seq_len = len(content_ids)

    q_ids = np.zeros(self.max_seq,dtype=int)
    ans = np.zeros(self.max_seq,dtype=int)
    r_time = np.zeros(self.max_seq,dtype=int)
    exe_cat = np.zeros(self.max_seq,dtype=int)

    if seq_len>=self.max_seq:
      q_ids[:] = content_ids[-self.max_seq:]
      ans[:] = answered_correctly[-self.max_seq:]
      r_time[:] = response_time[-self.max_seq:]
      exe_cat[:] = exe_category[-self.max_seq:]
    else:
      q_ids[-seq_len:] = content_ids
      ans[-seq_len:] = answered_correctly
      r_time[-seq_len:] = response_time
      exe_cat[-seq_len:] = exe_category
    
    decoder_input = np.insert(ans[:-1],0,0)
    label = ans[:]

    input_ids = np.zeros(self.max_seq,dtype=int)
    input_ids = q_ids[:].copy()

    input_rtime = np.zeros(self.max_seq,dtype=int)
    input_rtime = r_time[:].copy()

    input_cat = np.zeros(self.max_seq,dtype=int)
    input_cat = exe_cat[:].copy()

    input = {"input_ids":input_ids,"input_rtime":input_rtime.astype(np.int),"input_cat":input_cat}

    return input,decoder_input,label 

def get_dataloaders():              
    dtypes = {'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16',
          'answered_correctly':'int8',"prior_question_elapsed_time":"float32","task_container_id":"int16",
         'content_type_id':'bool'}
    print("loading csv.....")
    train_df = dt.fread(TRAIN_FILE, columns=set(dtypes.keys()), max_nrows=1000).to_pandas() # ,max_nrows=1000
    print("shape of dataframe :",train_df.shape) 

    train_df = train_df[train_df.content_type_id==0]
    train_df.prior_question_elapsed_time /= 1000
    train_df.prior_question_elapsed_time.fillna(300,inplace=True)
    train_df.prior_question_elapsed_time.clip(lower=0,upper=300,inplace=True)
    train_df.prior_question_elapsed_time = train_df.prior_question_elapsed_time.astype(np.int)
    
    
    train_df = train_df.sort_values(["timestamp"],ascending=True).reset_index(drop=True)
    skills = train_df.content_id.unique()
    n_skills = len(skills)
    n_cats = len(train_df.task_container_id.unique())+100
    print("no. of skills :",n_skills)
    print("no. of categories: ", n_cats)
    print("shape after exlusion:",train_df.shape)

    #grouping based on user_id to get the data supplu
    print("Grouping users...")
    group = train_df[["user_id","content_id","answered_correctly","prior_question_elapsed_time","task_container_id"]]\
                    .groupby("user_id")\
                    .apply(lambda r: (r.content_id.values,r.answered_correctly.values,r.prior_question_elapsed_time.values,r.task_container_id.values))
    del train_df
    gc.collect()

    print("splitting")
    train,val = train_test_split(group,test_size=VAL_RATE) 
    print("train size: ",train.shape,"validation size: ",val.shape)
    train_dataset = Dataset(train.values,n_skills=n_skills,max_seq = MAX_SEQ)
    val_dataset = Dataset(val.values,n_skills=n_skills,max_seq = MAX_SEQ)
    train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)
    val_loader = DataLoader(val_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=False)
    del train_dataset,val_dataset,group
    gc.collect()
    return train_loader, val_loader

# model building

In [4]:
import torch
import numpy as np
from torch import nn
import copy

def ut_mask(seq_len):
    """ Upper Triangular Mask
    """
    return torch.triu(torch.ones(seq_len,seq_len),diagonal=1).to(dtype=torch.bool)

def lt_mask(seq_len):
    """ Upper Triangular Mask
    """
    return torch.tril(torch.ones(seq_len,seq_len),diagonal=-1).to(dtype=torch.bool)

def pos_encode(seq_len):
    """ position Encoding
    """
    return torch.arange(seq_len).unsqueeze(0)

def get_clones(module, N):
    """ Cloning nn modules
    """
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [5]:
from torch import nn

#   MultiHead(Qin,Kin,Vin) = Concat(head1,··· ,headh)WO
class FFN(nn.Module):
  def __init__(self,features):
    super(FFN,self).__init__()
    self.layer1 = nn.Linear(features, features)
    self.layer2 = nn.Linear(features, features)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    
  def forward(self, x):
    out = self.drop(self.relu(self.layer1(x)))
    out = self.layer2(out)
    return out

class MultiHeadWithFFN(nn.Module):
  def __init__(self,n_heads,n_dims,mask_type="ut",dropout=0.2):
    super(MultiHeadWithFFN,self).__init__()
    self.n_dims = n_dims
    self.mask_type = mask_type
    self.multihead_attention = nn.MultiheadAttention(embed_dim = n_dims,
                                                      num_heads = n_heads,
                                                        dropout = dropout)
    self.layer_norm1 = nn.LayerNorm(n_dims)
    self.ffn = FFN(features = n_dims)
    self.layer_norm2 = nn.LayerNorm(n_dims)


  def forward(self,q_input,kv_input):
    q_input = q_input.permute(1,0,2)
    kv_input = kv_input.permute(1,0,2)
    query_norm = self.layer_norm1(q_input)
    kv_norm = self.layer_norm1(kv_input)
    if self.mask_type=="ut":
      mask = ut_mask(q_input.size(0))
    else: 
      mask = lt_mask(q_input.size(0))
    if device == "cuda":
      mask = mask.cuda()
    out_atten , weights_attent = self.multihead_attention(query=query_norm,
                                                key = kv_norm,
                                                value = kv_norm,
                                                attn_mask = mask)
    out_atten +=  query_norm
    out_atten = out_atten.permute(1,0,2)
    output_norm = self.layer_norm2(out_atten)
    output = self.ffn(output_norm)
    return output + output_norm 

In [6]:
import torch
from torch import nn

class SAINT(nn.Module):
    def __init__(self,n_encoder,n_decoder,enc_heads,dec_heads,n_dims,total_ex,total_cat,total_responses,seq_len):
        super(SAINT,self).__init__()
        self.n_encoder = n_encoder
        self.n_decoder = n_decoder
        self.enocder = get_clones(EncoderBlock(enc_heads,n_dims,total_ex,total_cat,seq_len),n_encoder)
        self.decoder = get_clones(DecoderBlock(dec_heads,n_dims,total_responses,seq_len),n_decoder)
        self.fc = nn.Linear(n_dims,1)
    
    def forward(self,in_exercise,in_category,in_response):
        first_block = True
        for n in range(self.n_encoder):
          if n>=1:
            first_block=False
          
          enc = self.enocder[n](in_exercise,in_category,first_block=first_block)
          in_exercise = enc
          in_category = enc

        first_block = True
        for n in range(self.n_decoder):
          if n>=1:
            first_block=False
          dec = self.decoder[n](in_response,encoder_output=in_exercise,first_block=first_block)
          in_exercise = dec
          in_response = dec
          
        return torch.sigmoid(self.fc(dec))



class EncoderBlock(nn.Module):
  def __init__(self,n_heads,n_dims,total_ex,total_cat,seq_len):
    super(EncoderBlock,self).__init__()
    self.seq_len = seq_len
    self.exercise_embed = nn.Embedding(total_ex,n_dims)
    self.category_embed = nn.Embedding(total_cat,n_dims)
    self.position_embed = nn.Embedding(seq_len,n_dims)
    self.layer_norm = nn.LayerNorm(n_dims)
    
    self.multihead = MultiHeadWithFFN(n_heads=n_heads,
                                            n_dims = n_dims)
  
  def forward(self,input_e,category,first_block=True):
    if first_block:
      if device == "cuda": 
        input_e = input_e.cuda() 
        category = category.cuda() 
        _exe = self.exercise_embed(input_e.long())
        _cat = self.category_embed(category.long())
        position_encoded = pos_encode(self.seq_len).cuda()
        _pos = self.position_embed(position_encoded.long())
        out = _cat + _exe + _pos
      else: 
        _exe = self.exercise_embed(input_e.long())
        _cat = self.category_embed(category.long())
        position_encoded = pos_encode(self.seq_len)
        _pos = self.position_embed(position_encoded.long())
        out = _cat + _exe + _pos
    else:
      out = input_e
    output = self.multihead(q_input=out,kv_input=out)
    return output


class DecoderBlock(nn.Module):
    def __init__(self,n_heads,n_dims,total_responses,seq_len):
      super(DecoderBlock,self).__init__()
      self.seq_len = seq_len
      self.response_embed = nn.Embedding(total_responses,n_dims)
      self.position_embed = nn.Embedding(seq_len,n_dims)
      self.layer_norm = nn.LayerNorm(n_dims)
      self.multihead_attention = nn.MultiheadAttention(embed_dim=n_dims,
                                            num_heads = n_heads,
                                            dropout = 0.2)
      self.multihead = MultiHeadWithFFN(n_heads=n_heads,
                                              n_dims = n_dims)

    def forward(self,input_r,encoder_output,first_block=True):
      if first_block:
        _response = self.response_embed(input_r.long())
        position_encoded = pos_encode(self.seq_len)
        if device == "cuda": 
          position_encoded = position_encoded.cuda()
        _pos = self.position_embed(position_encoded.long())
        out = _response + _pos
      else:
        out = input_r      
      out = out.permute(1,0,2)    
      #assert out_embed.size(0)==n_dims, "input dimention should be (seq_len,batch_size,dims)"
      out_norm = self.layer_norm(out)
      mask = ut_mask(out_norm.size(0))
      if device == "cuda":
        mask = mask.cuda()
      out_atten , weights_attent = self.multihead_attention(query=out_norm,
                                                  key = out_norm,
                                                  value = out_norm,
                                                  attn_mask = mask)
      out_atten +=  out_norm
      out_atten = out_atten.permute(1,0,2)
      output = self.multihead(q_input=out_atten,kv_input=encoder_output)
      return output



In [7]:
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score

class Model(pl.LightningModule):
  def __init__(self,model_args):
    super().__init__()
    self.model = SAINT(**model_args)
    
  def forward(self,exercise,category,response):#SAINT
    return self.model(exercise,category,response)#SAINT


  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=1e-3)
  
  def training_step(self,batch,batch_idx):
    inputs,decoder_input,target = batch
    output = self(inputs["input_ids"],inputs["input_cat"],decoder_input)#SAINT
    target_mask = (decoder_input != 0)
    output = torch.masked_select(output.squeeze(),target_mask)
    target = torch.masked_select(target,target_mask)
    loss = nn.BCEWithLogitsLoss()(output.float(),target.float())
    return {"loss":loss,"output":output,"target":target}
 
  def training_epoch_end(self, training_ouput):
    out = np.concatenate([i["output"].cpu().detach().numpy()
                          for i in training_ouput]).reshape(-1)
    labels = np.concatenate([i["target"].cpu().detach().numpy()
                             for i in training_ouput]).reshape(-1)
    auc = roc_auc_score(labels, out)
    self.print("train auc", auc)
    self.log("train_auc", auc)
  
  def validation_step(self,batch,batch_idx):
    inputs,decoder_input,target = batch
    output = self(inputs["input_ids"],inputs["input_cat"],decoder_input)#SAINT
    target_mask = (decoder_input != 0)
    output = torch.masked_select(output.squeeze(),target_mask)
    target = torch.masked_select(target,target_mask)
    loss = nn.BCEWithLogitsLoss()(output.float(),target.float())
    return {"val_loss":loss,"output":output,"target":target}

  def validation_epoch_end(self, validation_ouput):
    out = np.concatenate([i["output"].cpu().detach().numpy()
                          for i in validation_ouput]).reshape(-1)
    labels = np.concatenate([i["target"].cpu().detach().numpy()
                             for i in validation_ouput]).reshape(-1)
    auc = roc_auc_score(labels, out)
    self.print("val auc", auc)
    self.log("val_auc", auc)

# model train 

In [8]:
if __name__ == '__main__':
  train_loader, val_loader = get_dataloaders()

  ARGS = {'n_dims':EMBED_DIMS ,
          'n_encoder':NUM_ENCODER,
          'n_decoder':NUM_DECODER,
          'enc_heads':ENC_HEADS,
          'dec_heads':DEC_HEADS,
          'total_ex':TOTAL_EXE,
          'total_cat':TOTAL_CAT,
          'total_responses':TOTAL_RESP,
          'seq_len':MAX_SEQ}

#   checkpoint = ModelCheckpoint(filename="{epoch}_model",
#                                 verbose=True,
#                                 save_top_k=1,
#                                 monitor="val_loss")

  model = Model(model_args=ARGS)
  
  trainer = pl.Trainer(progress_bar_refresh_rate=1,
                       max_epochs=MAX_EPOCHS,
#                        callbacks=[checkpoint],)
                       gpus=1)
  print('Model training start')
  trainer.fit(model = model,train_dataloader=train_loader,val_dataloaders=val_loader) 
  trainer.save_checkpoint("saint_model_"+str(MAX_EPOCHS)+"_epochs.pt")

loading csv.....
shape of dataframe : (1000, 7)
no. of skills : 871
no. of categories:  396
shape after exlusion: (982, 7)
Grouping users...
splitting
train size:  (7,) validation size:  (1,)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Model training start



  | Name  | Type  | Params
--------------------------------
0 | model | SAINT | 32.9 M


val auc 0.35000000000000003


val auc 0.45000000000000007
train auc 0.5322274881516588


val auc 0.4
train auc 0.5570057115080812


val auc 0.4
train auc 0.5469680398590351


val auc 0.4
train auc 0.5417790740065622


val auc 0.4
train auc 0.5297362984566776



# model test

In [9]:
# ARGS = {'n_dims':EMBED_DIMS ,
#         'n_encoder':NUM_ENCODER,
#         'n_decoder':NUM_DECODER,
#         'enc_heads':ENC_HEADS,
#         'dec_heads':DEC_HEADS,
#         'total_ex':TOTAL_EXE,
#         'total_cat':TOTAL_CAT,
#         'total_responses':TOTAL_RESP,
#         'seq_len':MAX_SEQ}

# #   checkpoint = ModelCheckpoint(filename="{epoch}_model",
# #                                 verbose=True,
# #                                 save_top_k=1,
# #                                 monitor="val_loss")

# model = Model(model="saint",model_args=ARGS)
# model = model.load_from_checkpoint('../input/yrquni-riiid/lightning_logs/version_0/checkpoints/epoch=4.ckpt',model_args=ARGS)

In [10]:
# model.eval().to(device)

# test data

In [11]:
# import gc
# import datatable as dt
# import numpy as np
# import pandas as pd
 
# from torch.utils.data import Dataset, DataLoader
# import gc
# from sklearn.model_selection import train_test_split 


# class TestDataset(Dataset):
#   def __init__(self,group,n_skills,max_seq = 100):
#     self.samples = group
#     self.n_skills = n_skills
#     self.max_seq = max_seq
#     self.data = []

#     for user_id,buf in self.samples:
#         que,ans,res_time,exe_cat,row_id = buf
#         if len(que)>=self.max_seq:
#             self.data.extend([(user_id,que[-self.max_seq:],ans[-self.max_seq:],res_time[-self.max_seq:],exe_cat[-self.max_seq:],row_id[-self.max_seq:])])
#         elif len(que)<self.max_seq and len(que)>10:
#             self.data.append((user_id,que,ans,res_time,exe_cat,row_id))
#         else :
#             self.data.append((user_id,que,ans,res_time,exe_cat,row_id))
  
#   def __len__(self):
#     return len(self.data)
  
#   def __getitem__(self,idx):
#     user_id,content_ids,answered_correctly,response_time,exe_category,row_id = self.data[idx]
#     seq_len = len(content_ids)

#     q_ids = np.zeros(self.max_seq,dtype=int)
#     ans = np.zeros(self.max_seq,dtype=int)
#     r_time = np.zeros(self.max_seq,dtype=int)
#     exe_cat = np.zeros(self.max_seq,dtype=int)
#     row_id_  = np.zeros(self.max_seq,dtype=int)
    
    
#     if seq_len>=self.max_seq:
#       q_ids[:] = content_ids[-self.max_seq:]
#       ans[:] = answered_correctly[-self.max_seq:]
#       r_time[:] = response_time[-self.max_seq:]
#       exe_cat[:] = exe_category[-self.max_seq:]
#       row_id_[:] = row_id[-self.max_seq:]
#     else:
#       q_ids[-seq_len:] = content_ids
#       ans[-seq_len:] = answered_correctly
#       r_time[-seq_len:] = response_time
#       exe_cat[-seq_len:] = exe_category
#       row_id_[-seq_len:] = row_id
    
#     decoder_input = np.insert(ans[:-1],0,0)
#     label = ans[:]

#     input_ids = np.zeros(self.max_seq,dtype=int)
#     input_ids = q_ids[:].copy()

#     input_rtime = np.zeros(self.max_seq,dtype=int)
#     input_rtime = r_time[:].copy()

#     input_cat = np.zeros(self.max_seq,dtype=int)
#     input_cat = exe_cat[:].copy()
    
#     inputs = {"input_ids":input_ids,"input_rtime":input_rtime.astype(np.int),"input_cat":input_cat}

#     row_id_test = np.zeros(self.max_seq,dtype=int)
#     row_id_test = row_id_[:].copy()
    
#     user_id_test = np.zeros(self.max_seq,dtype=int)
#     user_id_test = np.repeat(user_id, self.max_seq, axis=0)

#     return inputs,decoder_input,label,row_id_test,user_id_test

# def get_Test_dataloaders(train_df,test_df,test_user_id):    
    
#     dtypes = {'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16',
#           'answered_correctly':'int8',"prior_question_elapsed_time":"float32","task_container_id":"int16",
#          'content_type_id':'bool','row_id':'int'}
#     train_df = train_df.loc[train_df.user_id.isin(test_user_id)]
#     test_df = train_df.append(test_df.loc[test_df.user_id.isin(test_user_id)],dtypes.keys())

#     test_df = test_df[test_df.content_type_id==0]
#     test_df.prior_question_elapsed_time /=1000
#     test_df.prior_question_elapsed_time.fillna(300,inplace=True)
#     test_df.prior_question_elapsed_time.clip(lower=0,upper=300,inplace=True)
#     test_df.prior_question_elapsed_time = test_df.prior_question_elapsed_time.astype(np.int)
      
#     test_df = test_df.sort_values(["timestamp"],ascending=True).reset_index(drop=True)
#     skills = test_df.content_id.unique()
#     n_skills = len(skills)
#     n_cats = len(test_df.task_container_id.unique())+100

#     #grouping based on user_id to get the data supplu
#     group = test_df[["user_id","content_id","answered_correctly","prior_question_elapsed_time","task_container_id",'row_id']]\
#                     .groupby("user_id")\
#                     .apply(lambda r: (r.content_id.values,r.answered_correctly.values,r.prior_question_elapsed_time.values,r.task_container_id.values,r.row_id.values)).reset_index()
#     del train_df,test_df
#     gc.collect()
#     test_dataset = TestDataset(group.values,n_skills=n_skills,max_seq = MAX_SEQ)
#     test_loader = DataLoader(test_dataset,
#                           batch_size=TEST_BATCH_SIZE,
#                           num_workers=4,
#                           shuffle=False)
#     del test_dataset, group
#     gc.collect()
#     return test_loader

# strat test loop

In [12]:
# import riiideducation

# env = riiideducation.make_env()
# iter_test = env.iter_test()

In [13]:
# import psutil
# prev_test_df = None
# test_df_all = None

# dtypes = {'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16',
#           'answered_correctly':'int8',"prior_question_elapsed_time":"float32","task_container_id":"int16",
#          'content_type_id':'bool','row_id':'int'}
# print("loading csv.....")
# train_df = dt.fread(TRAIN_FILE, columns=set(dtypes.keys()), ).to_pandas()#,max_nrows=1000

In [14]:
# BUF = pd.DataFrame()

# for (test_df, sample_prediction_df) in iter_test:
#     print('test_df user_id nums: ',len(test_df.copy().user_id.values))
#     test_user_id = test_df.copy().user_id.unique()
#     if (prev_test_df is not None) & (psutil.virtual_memory().percent < 90):
#         test_df['answered_correctly'] = -1
#         prev_test_df.loc[prev_test_df.answered_correctly==-1,'answered_correctly'] = eval(test_df['prior_group_answers_correct'].copy().iloc[0])
#         prev_test_df = prev_test_df.append(test_df[dtypes.keys()].copy()).copy()
#         test_df_all = prev_test_df.copy()
#         test_df_all['answered_correctly'].replace(-1,0,inplace=True)
   
#     else:
#         test_df['answered_correctly'] = -1
#         prev_test_df = test_df.copy()
#         test_df_all = test_df.copy()
#         test_df_all['answered_correctly'].replace(-1,0,inplace=True)
        
#     test_df_all = test_df_all[test_df_all.content_type_id == False]
#     test_loader = get_Test_dataloaders(train_df,test_df_all,test_user_id)
   
#     ans = []
#     user_id_buf = []
#     row_id_test_buf = []
#     for inputs,decoder_input,label,row_id_test,user_id_test in test_loader:
#         outs = model(inputs["input_ids"].flatten().view(-1,MAX_SEQ).to(device),
#                      inputs["input_cat"].flatten().view(-1,MAX_SEQ).to(device),
#                      decoder_input.flatten().view(-1,MAX_SEQ).to(device))
#         ans.extend(torch.sigmoid(outs).view(-1).data.cpu().numpy())
#         user_id_buf.extend(user_id_test.view(-1).data.cpu().numpy())
#         row_id_test_buf.extend(row_id_test.view(-1).data.cpu().numpy())
       
#     ans_df_buf = pd.DataFrame({'user_id':user_id_buf,'row_id':row_id_test_buf,'answered_correctly':ans})
    
#     test_df = pd.merge(test_df.drop(columns=['answered_correctly']),ans_df_buf,on=['user_id','row_id'],how='left')
#     env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
    
#     BUF = BUF.append(test_df)

# preview the submit DataFrame

In [15]:
# BUF.loc[BUF['content_type_id'] == 0, ['row_id', 'answered_correctly']].answered_correctly.isnull().any()